In [113]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from urllib import parse
from tqdm import tqdm

In [2]:
url = 'https://movie.naver.com/movie/point/af/list.naver?&page=1'
html = requests.get(url)
soup = bs(html.text, 'html.parser')

In [47]:
movie_rating = soup.select('table.list_netizen > tbody')

In [48]:
movie = []

for item in movie_rating:
    number = item.find_all('td', {'ac num'})[0].text
    title = item.find_all('td', {'title'})[0].text
    rating = item.find_all('sapn', {'st_on'})[0].text

    movie.append([number,title, rating])

movie



[]

In [102]:
# for x in range(1,101):
naver_mov = f'https://movie.naver.com/movie/point/af/list.naver?&page=1'
naver_req = requests.get(url)
table_pd = pd.read_html(naver_req.text)

In [118]:
table_pd[0]['감상평'][0]

'우리도 사랑일까  별점 - 총 10점 중8  지극히 평범한 권태에 대한 이야기. 결국은 가져가야 하는 것. 그 모든 것에 대한 이야기. 신고'

In [122]:
table_pd[0]['감상평'][1]

'아바타: 물의 길  별점 - 총 10점 중10  오늘 14,000원 주고 본 영화... 영화가 아깝다는 생각을 처음했다..돈 더주고 봐야하는 영화. 내가 이 영화를 즐길수 있다는게 너무행복했다 신고'

In [119]:
table_pd[0]['감상평'][0].split("  별점 - 총 10점 중")

['우리도 사랑일까', '8  지극히 평범한 권태에 대한 이야기. 결국은 가져가야 하는 것. 그 모든 것에 대한 이야기. 신고']

In [120]:
            table_pd[0]['감상평'][0].split("  별점 - 총 10점 중")[0]

'우리도 사랑일까'

In [121]:
table_pd[0]['감상평'][0].split("  별점 - 총 10점 중")[1]

'8  지극히 평범한 권태에 대한 이야기. 결국은 가져가야 하는 것. 그 모든 것에 대한 이야기. 신고'

In [157]:
# 정리 (속도개선)
import pandas as pd
import requests
from tqdm import tqdm
from multiprocessing import Pool
import multiprocessing
import datetime
import warnings
warnings.simplefilter(action="ignore")

multiprocessing.get_start_method()
from multiprocessing import get_context


def collect_naver_review(page):
    df = pd.DataFrame(columns=["번호", "제목", "평점", "감상평"])
    naver_mov = f"https://movie.naver.com/movie/point/af/list.naver?&page={page}"
    naver_req = requests.get(naver_mov)
    table_pd = pd.read_html(naver_req.text)

    for i in range(10):
        try:      # try except는 예외 처리시 사용
            name = table_pd[0]["감상평"][i].split("  별점 - 총 10점 중")[0]
            data = table_pd[0]["감상평"][i].split("  별점 - 총 10점 중")[1][:-3].split("  ")
            index = table_pd[0]["번호"][i]
            if len(data) == 2:
                df = df.append({"번호":int(index), "제목": name, "평점": int(data[0]), "감상평": data[1]}, ignore_index=True)
        except:
            pass
    return df


pool = get_context("fork").Pool(processes=4)
inputs = range(1,100)

results = []
# imap_unordered vs map ??

for result in tqdm(pool.imap_unordered(collect_naver_review, inputs), total=len(inputs)):
    results.append(result)

# for result in tqdm(pool.map(collect_naver_review, inputs), total=len(inputs)):
#     results.append(result)
    
naver_df = pd.concat(results, axis=0).reset_index()
naver_df.to_csv(f"{datetime.date.today()}.csv")

naver_df


100%|██████████| 99/99 [00:03<00:00, 30.19it/s]


,index,번호,제목,평점,감상평
0,0,18595145,아바타: 물의 길,1,진짜 오직 볼건 영상미가 끝이다. 다큐멘터리 3시간이다.스토리? 기대하지도 마셈. ...
1,1,18595144,아바타: 물의 길,10,무조건 3D 아맥으로 봐야됨
2,2,18595143,아바타: 물의 길,5,영상미는 역시나 훌륭하고 볼거리를 제공함.스토리나 재미는 없고 많이 지루함.. 특히...
3,3,18595142,"오늘 밤, 세계에서 이 사랑이 사라진다 해도",10,"비록 제가 울진 않았지만, 연말에 보기좋은 소중한 영화예요"
4,4,18595141,보디가드,8,그때 그 시절의 감성이 믈씬
...,...,...,...,...,...
944,3,18594149,아바타: 물의 길,4,"1편에서 너무 신선한 충격을 받아서일까요,13년을 손꼽아 기다렸는데 영상미는 훌륭했..."
945,4,18594147,아바타: 물의 길,10,그냥 다 쩔었다! 오랜만에 완벽에 가까운 영화 본 느낌! 3부 잘 마무리해줘!
946,5,18594146,아바타: 물의 길,10,어려운 시기에 훌륭한 영화 만들어 주셔서 감사드립니다. 아름답습니다.
947,6,18594145,아바타: 물의 길,8,CG로 표현할수있는 최대치무난한 스토리


In [ ]:
# 기존꺼


df = pd.DataFrame(columns=['번호','제목','평점','감상평'])

for j in tqdm(range(100)):
    naver_mov = f'https://movie.naver.com/movie/point/af/list.naver?&page={j+1}'
    naver_req = requests.get(url)
    table_pd = pd.read_html(naver_req.text)


    for i in range(10):
        name = table_pd[0]['감상평'][i].split("  별점 - 총 10점 중")[0]
        data = table_pd[0]['감상평'][i].split("  별점 - 총 10점 중")[1][:-3].split('  ')
        index = table_pd[0]['번호'][i]
        if len(data) == 2:
            df = df.append({'번호':int(index), '제목':name, '평점': int(data[0]), '감상평': data[1]}, ignore_index= True)

df
       

100%|██████████| 100/100 [00:12<00:00,  8.00it/s]


,번호,제목,평점,감상평
0,18594106,아바타: 물의 길,10,190분동안의 러닝타임이였지만...1초도 지루하지않았고 너무너무 감동적이였다
1,18594105,아바타: 물의 길,10,후반부 세월호사건이 연상 되었는데그장면을 아름답게 풀어내는 장면에서가슴아프고 먹먹했...
2,18594104,아바타: 물의 길,8,영상미 10스토리7총점8점극장에서추천
3,18594103,아바타: 물의 길,10,내가뭘본건가....영화라부를수없는..
4,18594102,아바타: 물의 길,4,조조로 보러갔는데 아침 일찍 일어나서 그런지 2시간 이후부터 계속 졸았습니다. 영상...
...,...,...,...,...
995,18594101,아바타: 물의 길,10,말이 필요없다. 최고다.
996,18594100,아바타: 물의 길,9,판도라 행성을 배경으로 한 내셔널지오그래픽다큐멘터리!!
997,18594099,아바타: 물의 길,6,평점 낮게 줬다고 테러를 하네 ㅋㅋ 하여간 국민성 알아줘야되.. 일단 cg랑 비쥬얼...
998,18594098,아바타: 물의 길,8,"아바타1에 비해서 감동은 덜했지만,영상미 대박이고 물의 길 다음이야기는 어떨지 궁금하네요"
